# Initiation à Hadoop et Map-Reduce

L'Objectif du TP: 
Initiation au framework hadoop et au patron MapReduce, utilisation de docker pour lancer un cluster hadoop.
Les consignes de ce notebook sont à réaliser dans votre cluster deployé avec docker, mais vous pouvez utiliser le notebook pour stocker les commandes que vous avez utilisé afin de réaliser une sorte de cheat sheet.

- Clonez le repo à l’adresse ?? dans votre répertoire de travail.


- Démarrez votre image docker: \
Ouvrez votre terminal et placez-vous dans le dossier ?? fraîchement récupéré. \
Lancez la commande: \
	docker-compose up -d


- Connectez vous au namenode de votre cluster:\
Lancez la commande: \
    docker exec -it namenode bash


Refs: \
Apache Hadoop [http://hadoop.apache.org/] \
Docker [https://www.docker.com/]



- Placez vous dans le dossier utilisateur: \
cd ~


- Créer un répertoire TP, puis deux sous-répertoires code et data dans lesquels sauvegarder respectivement les codes des mappers et reducers, et les données sources et résultats.


- En local (en dehors de docker), avec la commande docker adequate envoyez le fichier ventes.txt fourni, dans le répertoire ~/TP/data du namenode.

### 1- Hadoop HDFS
   
##### Les commandes les plus utilisées dans Hadoop: 
hadoop fs –ls             Afficher le contenu du répertoire racine

hadoop fs –put file.txt   Upload un fichier dans hadoop (à partir du répertoire courant linux)

hadoop fs –get file.txt   Download un fichier à partir de hadoop sur votre disque local

hadoop fs –tail file.txt  Lire les dernières lignes du fichier

hadoop fs –cat file.txt   Affiche tout le contenu du fichier

hadoop fs –mv file.txt newfile.txt   Renommer le fichier

hadoop fs –rm newfile.txt            Supprimer le fichier

hadoop fs –mkdir myinput             Créer un répertoire

hadoop fs –cat file.txt | less       Lire le fichier page par page

- Créez un répertoire dans HDFS, appelé data.


- Envoyez le fichier ventes.txt depuis le dossier ~/TP/data vers le répertoire data HDFS.


- Affichez le contenu du répertoire data HDFS et visualisez les dernières lignes du fichier.

In [ ]:
# Créer le rep data
hadoop fs -mkdir /data
#si vous avez des soucis de safe mode lancer : sudo -u hdfs hdfs dfsadmin -safemode leave et relancer ensuite mkdir!

#Charger le fichier sur HDFS
# envoyer le fichier avec docker : docker cp /host/pathto/ventes.txt <containerId>:/container/path/

hadoop fs -put ventes.txt /data/
hadoop fs -ls /data
hadoop fs -tail /data/ventes.txt



### 2-  Map Reduce
Un Job Map-Reduce se compose principalement de deux types de programmes:

Mappers : permettent d’extraire les données nécessaires sous forme de clef/valeur, pour pouvoir ensuite appliquer le shuffle &sort. \
Reducers : prennent un ensemble de données triées , et effectuent le traitement nécessaire sur ces données (somme, moyenne,total...)

PS : Vous aurez besoin d'installer python dans votre namenode docker avec apt-get pour tester vos mapper et reducer.

##### 2.1 )

Soit le dataset ventes.txt comportant 6 champs, séparés par des tabulations, de la forme suivante:

date temps magasin produit coût paiement

Le but est de déterminer le total des ventes par magasin.

- Créez un fichier mapper.py dans le dossier ~/TP/code , qui permet de: \
Séparer les différents champs par tabulation. \
Extraire les éléments à partir de ces champs, sous forme de clé/valeur (magasin,coût), Pour calculer les ventes par magasin.

In [ ]:
#mapper.py
import sys
for line in sys.stdin:
    data = line.strip().split('\t')
    if len(data) == 6:
        date, time, store, item, cost, payment = data
        print ('{0}\t{1}'.format(store,cost))

- Testez votre mapper sur les 50 premières lignes du fichier ventes.txt 

In [ ]:
head -50 ventes.txt | python ./mapper.py echo

##### 2.2 )
Le Reducer permet de faire le traitement définit sur des entrées préalablement triées par Hadoop sur les couples (magasin,coût). \
Le Reducer fera la somme de tous les coûts pour un même magasin. 

- Créez un fichier reducer.py dans le dossier ~/TP/code.


In [ ]:
#reducer.py
import sys
salesTotal = 0
oldKey = None
for line in var:
	data = line.strip().split('\t')
	if len(data) != 2:
		continue
	thisKey, thisSale = data
	if oldKey and oldKey != thisKey:
		print ('{0} {1}'.format(oldKey,salesTotal))
		oldKey = thisKey;
		salesTotal = 0
	oldKey = thisKey
	salesTotal += float(thisSale)
if oldKey != None:
    print ('{0} {1}'.format(oldKey, salesTotal))


- Testez votre Reducer sur les 50 premières lignes du fichier ventes.txt

In [ ]:
head -50 ventes.txt | python ./mapper.py |sort | python ./reducer.py

##### 2.3)
Lancer un job entier sur Hadoop =>  faire appel au mapper puis au reducer sur une entrée volumineuse, et obtenir à la fin un résultat, directement sur HDFS.

- Exécutez un job hadoop sur le fichier ventes.txt en utilisant les fichiers mapper.py et reducer.py déjà implémentés. 
- Stocker le résultat dans un répertoire joboutput.


- Sauvegarder ensuite le fichier part-00000 dans le dossier ~/TP/data.

In [ ]:
hadoop jar /opt/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar --mapper mapper.py --reducer reducer.py --file /root/TP/code/mapper.py --file /root/TP/code/reducer.py --input /data --output /joboutput1

In [ ]:
hdfs dfs -get /data/part-00000....

- Quelle est la totalité des ventes du magasin de Buffalo ?

- Créez un nouveau mapper/reducer pour trouver le total des ventes par catégorie de produits.

In [ ]:
#mapper2.py
import sys

for line in sys.stdin:
    data = line.strip().split("\t")
    if len(data) == 6:
        date, time, store, item, cost, payment = data
print ("{0}\t{1}".format(item, cost))

In [ ]:
#Le reducer NE CHANGE PAS!

- Exécutez un job hadoop sur le fichier ventes.txt en utilisant les fichiers mapper2.py et reducer.py 

- Quelle est la valeur des ventes pour la catégorie Toys?


- Et pour la catégorie Consumer Electronics?

##### Bonus

- Créez un nouveau mapper/reducer pour faire la liste des montants de ventes pour chaque magasin.


In [ ]:
#mapper3.py

In [ ]:
#Reducer2.py
import sys

salesMax = 0
oldKey = None

for line in sys.stdin:
    data_mapped = line.strip().split("\t")
    if len(data_mapped) != 2:
        # Something has gone wrong. Skip this line.
        continue

    thisKey, thisSale = data_mapped

    if oldKey and oldKey != thisKey:
        print oldKey, "\t", salesMax
        oldKey = thisKey;
        salesMax = 0

    oldKey = thisKey
    salesMax = max(salesMax, float(thisSale))

if oldKey != None:
print oldKey, "\t", salesMax

- Quelle est la valeur de la vente la plus élévée pour les magasins suivants:

o Reno

o Toledo

o Chandler


- Quel est le nombre total des ventes et la valeur totale des ventes de tous magasins confondus?